This notebook contains the commands to export selected visualizations as HTML.

Just run this notebook to save the visualizations as interactive HTML maps. Use the [visualize](./visualize.ipynb) notebook if you want to learn how to create dataframes and visualizations from PostGIS data.

In [1]:
import sys
from ipygis import get_connection_url, QueryResult, get_map, generate_map
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from kepler_h3_config import config  # we may use our own custom visualization config

In [2]:
sql_url = get_connection_url(dbname='geoviz')
engine = create_engine(sql_url)
session = sessionmaker(bind=engine)()

sys.path.insert(0, '..')
from models import OSMPoint, OSMPolygon, FlickrPoint, GTFSStop, OSMAccessNode, OoklaPoint, KonturPoint
from osm_tags import tag_filter

In [3]:
places_query = session.query(OSMPoint).filter(tag_filter)
photos_query = session.query(FlickrPoint)
stops_query = session.query(GTFSStop)
access_points_query = session.query(OSMAccessNode)
ookla_query = session.query(OoklaPoint)
kontur_query = session.query(KonturPoint)

You may visualize all points in any database query by simply

In [4]:
get_map(ookla_query)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'data_1', 'type': '…

Datasets
--------

The next phase will take a while. Here we actually create all the dataframes in memory (using H3 hex resolution 8), from the whole database.

The **places** dataset simply sums the *number of amenity POIs* in each H3 hex. The larger areas you have imported, the longer this query will take:

In [4]:
places = QueryResult.create(places_query, resolution=8, name='places')

The **photographers** dataset calculates the *number of people taking photos* in each H3 hex. Grouping the points by hex and photographer takes a while:

In [5]:
photographers = QueryResult.create(photos_query, resolution=8, name='photographers', group_by='properties.owner')

The **trips** dataset calculates the *number of trips leaving from all transit stops* in the H3 hex in a single day. This shouldn't take too long:

In [6]:
trips = QueryResult.create(stops_query, resolution=8, name='trips', plot='sum', column='properties.ntrips')

The **access** dataset calculates the *average walking time to 5th closest amenity* in each H3 hex. The amount of walking nodes is quite big so this is a slow query:

In [7]:
access = QueryResult.create(access_points_query, resolution=8, name='access', plot='mean', column='accessibilities.5')

The **devices** dataset calculates the *number of broadband devices* making speedtests in each H3 hex. (Alternatively, we could consider the average broadband speeds in the hex.) This should be fast:

In [8]:
speeds = QueryResult.create(ookla_query, resolution=8, name='speeds', plot='mean', column='properties.avg_d_kbps')
devices = QueryResult.create(ookla_query, resolution=8, name='devices', plot='sum', column='properties.devices')

The **population** dataset contains the *approximate population* in the H3 hex. This should be fast:

In [9]:
population = QueryResult.create(kontur_query, resolution=8, name='population', plot='sum', column='properties.population')

Visualization
-------------

Once the QueryResults exist, calculating the sum map from dataframes should be relatively fast. Layer weights in the sum may be customized here if you want to emphasize or leave out some datasets.

(Note that access layer has *negative* weight, because smaller walking times indicate better pedestrian access to amenities.)

In [11]:
results = [places, photographers, trips, access, devices, population]
columns = ['size', 'size', 'ntrips', '5', 'devices', 'population']
weights = [1, 1, 1, -1, 1, 1]

In [7]:
result_map = generate_map(results, 500, config=config, column=columns, weights=weights)
result_map.save_to_html()
result_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to keplergl_map.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'hex8_places', 'typ…